In [6]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [118]:
driver = webdriver.Chrome()

names, prices, num_products, ratings_overall, solds = [], [], [], [], []
ratings, reviews, num_products1 = [], [], []
table_data = {}

url = 'https://www.tokopedia.com/officialinfinix/etalase/note-series'
driver.get(url)

# Scroll to load products
for scroll in range(6):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

products = 1
pro = 1
for x in soup.find_all('div', {'class': 'css-1sn1xa2'}):
    try:
        # Get product name and price
        name = x.find('div', {'class': 'prd_link-product-name css-3um8ox'})
        price = x.find('div', {'class': 'prd_link-product-price css-h66vau'})
        rating_overall = x.find('span', {'class' : 'prd_rating-average-text css-t70v7i'})
        sold = x.find('span', {'class': 'prd_label-integrity css-1sgek4h'})

        names.append(name.get_text() if name else None)
        prices.append(price.get_text() if price else None)
        ratings_overall.append(rating_overall.get_text() if rating_overall else None)
        solds.append(sold.get_text() if sold else None)
        num_products.append(products)
        products += 1
        
        # Navigate to product page
        link = x.find('a')['href']
        
        if link:
            driver.get(link)
            
            # Scroll reviews
            for scroll in range(20):
                driver.execute_script("window.scrollBy(0,300)")
                time.sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # Ratings and reviews
            for z in soup.find_all('div', {'class': "css-1k41fl7"}):
                rating = z.find('div', {'data-testid': 'icnStarRating'})
                review = z.find('span', {'data-testid': 'lblItemUlasan'})
                
                ratings.append(rating.get("aria-label") if rating else None)
                reviews.append(review.get_text() if review else None)
                num_products1.append(pro)


            pro += 1
            
    
    except Exception as e:
        print(f"Error: {e}")
        continue

driver.quit()
print('Scraping complete.')


Scraping complete.


In [119]:
print(len(names))
print(len(prices))
print(len(num_products))
print(len(ratings_overall))
print(len(solds))

8
8
8
8
8


In [120]:
data = pd.DataFrame({'product_id':num_products,'name':names, 'price':prices, 'rating':ratings_overall, 'sold':solds})
data.head()

,product_id,name,price,rating,sold
0,1,Infinix Note 40 Pro+ 5G 12/256GB - Up to 24GB ...,Rp3.949.000,4.9,250+ terjual
1,2,Infinix Note 40S 8/256GB - Up to 16GB Extended...,Rp3.099.000,4.9,500+ terjual
2,3,Infinix Note 40 Pro 5G 8/256GB - Up to 16GB Ex...,Rp3.799.000,4.9,1rb+ terjual
3,4,Infinix Note 40 Pro 8/256GB - Up to 16GB Exten...,Rp3.499.000,4.9,2rb+ terjual
4,5,Infinix Note 40 8/256GB - Up to 16GB Extended ...,Rp2.999.000,4.9,2rb+ terjual


In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  8 non-null      int64 
 1   name        8 non-null      object
 2   price       8 non-null      object
 3   rating      8 non-null      object
 4   sold        8 non-null      object
dtypes: int64(1), object(4)
memory usage: 452.0+ bytes


In [122]:
print(len(ratings))
print(len(reviews))
print(len(num_products1))

400
400
400


In [123]:
data1 = pd.DataFrame({'product_id':num_products1, 'rating':ratings, 'review':reviews})
data1.head()

,product_id,rating,review
0,1,bintang 4,"sesuai pesanan, semoga awet"
1,1,bintang 5,mantap lah utk brg ori dan segel. casing bawaa...
2,1,bintang 5,paking aman dan rapi \ndapat discount pula
3,1,bintang 4,"Dalam hal pengiriman, barangnya sampai dirumah..."
4,1,bintang 5,"penjual oke mantap, gercep bantu ketika ada ke..."


In [124]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  400 non-null    int64 
 1   rating      400 non-null    object
 2   review      384 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.5+ KB


In [125]:
data1.product_id.value_counts()

product_id
1    50
2    50
3    50
4    50
5    50
6    50
7    50
8    50
Name: count, dtype: int64

In [126]:
data.to_csv('infinix_analyst.csv', index=False)
data1.to_csv('infinix_scientist.csv', index=False)